In [1]:
import keras
from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
import numpy as np
from keras.layers.normalization import BatchNormalization
import functools
from keras import regularizers
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint
keras.__version__

Using TensorFlow backend.


'2.1.6'

In [2]:
net = keras.applications.mobilenet.MobileNet(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3))

In [3]:
x = net.output
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu', kernel_regularizer=regularizers.l1(0.), activity_regularizer=regularizers.l1(0.))(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(20)(x)
output_layer = Activation('softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)

In [4]:
count=0
for layer in net_final.layers:
    count+=1
print(count)

103


In [5]:
print(net_final.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

In [6]:
for layer in net_final.layers:
    layer.trainable = True

In [7]:
net_final.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
V1 = np.load("C:/Users/cloudadavncenew0709/Desktop/numpydataKeras_20_new_val/imgonehot_new_val_2000.npy")
V2 = np.load("C:/Users/cloudadavncenew0709/Desktop/numpydataKeras_20_new_val/labelonehot_new_val_2000.npy")

In [9]:
weights_file = 'C:/Users/cloudadavncenew0709/Desktop/MobileNet_Condition01_SaveBest_03.h5'
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='auto', verbose=2)
model_checkpoint = ModelCheckpoint(weights_file, monitor='val_loss', save_best_only=True, verbose=2)
callbacks = [early_stopping, model_checkpoint]

In [10]:
output = []

In [11]:
for i in range(5):#訓練5週期
    for j in range(100):#100,000張分100批訓練
        X = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/imgonehot_"+str((j+1)*1000)+".npy" )
        Y = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/labelonehot_"+str((j+1)*1000)+".npy" )
        net_final.fit(x = X, y = Y,
                      epochs = 1,
                      validation_data=(V1, V2),
                      callbacks = callbacks,
                      verbose = 1)
        loss = net_final.history.history['loss'][0]
        acc = net_final.history.history['acc'][0]
        val_loss = net_final.history.history['val_loss'][0]
        val_acc = net_final.history.history['val_acc'][0]
        epoch = float(i) + (float(j+1)/100.00)
        output.append([loss, acc, val_loss, val_acc, epoch])
        print(output[-1])
        print('Done training ', (j+1)*1000 ,' images in Epoch : ', i)
    print('Done training 100000 images, Epoch ', i ,'  -------------')
output_df = pd.DataFrame(output, columns=['loss', 'acc', 'val_loss', 'val_acc', 'epoch'])
output_df.to_csv("C:/Users/cloudadavncenew0709/Desktop/MobileNet_Condition01_SaveBest_data_new.csv")

Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 47s 47ms/step - loss: 2.8807 - acc: 0.1960 - val_loss: 2.3693 - val_acc: 0.2955

Epoch 00001: val_loss improved from inf to 2.36928, saving model to C:/Users/cloudadavncenew0709/MobileNet_Condition01_SaveBest_03.h5
[2.8806714076995847, 0.196, 2.3692802085876465, 0.2955, 0.01]
Done training  1000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 26s 26ms/step - loss: 2.2964 - acc: 0.3040 - val_loss: 2.0883 - val_acc: 0.3615

Epoch 00001: val_loss improved from 2.36928 to 2.08832, saving model to C:/Users/cloudadavncenew0709/MobileNet_Condition01_SaveBest_03.h5
[2.296438599586487, 0.304, 2.088323881149292, 0.3615, 0.02]
Done training  2000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 2.1400 - acc: 0.3790 - val_lo

[1.5654531211853027, 0.524, 1.550992664337158, 0.5075, 0.21]
Done training  21000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 26s 26ms/step - loss: 1.5476 - acc: 0.5430 - val_loss: 1.5576 - val_acc: 0.5085

Epoch 00001: val_loss did not improve from 1.55099
[1.547598871231079, 0.543, 1.5576102075576783, 0.5085, 0.22]
Done training  22000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 26s 26ms/step - loss: 1.5482 - acc: 0.5200 - val_loss: 1.7684 - val_acc: 0.4485

Epoch 00001: val_loss did not improve from 1.55099
[1.5482010612487793, 0.52, 1.768350706100464, 0.4485, 0.23]
Done training  23000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 1.5594 - acc: 0.5250 - val_loss: 1.6685 - val_acc: 0.4725

Epoch 00001: val_loss did not impro

Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 26s 26ms/step - loss: 1.3784 - acc: 0.5880 - val_loss: 1.5660 - val_acc: 0.5120

Epoch 00001: val_loss did not improve from 1.46992
[1.3783540806770325, 0.588, 1.56595011138916, 0.512, 0.45]
Done training  45000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 26s 26ms/step - loss: 1.3936 - acc: 0.5510 - val_loss: 1.5086 - val_acc: 0.5255

Epoch 00001: val_loss did not improve from 1.46992
[1.3936435222625732, 0.551, 1.5085815477371216, 0.5255, 0.46]
Done training  46000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 26s 26ms/step - loss: 1.4186 - acc: 0.5860 - val_loss: 1.4668 - val_acc: 0.5360

Epoch 00001: val_loss improved from 1.46992 to 1.46682, saving model to C:/Users/cloudadavncenew0709/MobileNet_Condition01_SaveBest_03.h5
[

1000/1000 [==============================] - 26s 26ms/step - loss: 1.3168 - acc: 0.5870 - val_loss: 1.4452 - val_acc: 0.5390

Epoch 00001: val_loss did not improve from 1.41298
[1.3167514133453369, 0.587, 1.4452442464828492, 0.539, 0.68]
Done training  68000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 1.3313 - acc: 0.5960 - val_loss: 1.4354 - val_acc: 0.5400

Epoch 00001: val_loss did not improve from 1.41298
[1.3312941513061523, 0.596, 1.4353513736724854, 0.54, 0.69]
Done training  69000  images in Epoch :  0
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 26s 26ms/step - loss: 1.3597 - acc: 0.5720 - val_loss: 1.4210 - val_acc: 0.5545

Epoch 00001: val_loss did not improve from 1.41298
[1.3596678256988526, 0.572, 1.4210300092697143, 0.5545, 0.7]
Done training  70000  images in Epoch :  0
Train on 1000 samples, validate on 2000 sa

Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 26s 26ms/step - loss: 1.2582 - acc: 0.6100 - val_loss: 1.4553 - val_acc: 0.5305

Epoch 00001: val_loss did not improve from 1.34828
[1.2582379550933838, 0.61, 1.4552672262191773, 0.5305, 1.15]
Done training  15000  images in Epoch :  1
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 26s 26ms/step - loss: 1.2240 - acc: 0.5930 - val_loss: 1.4191 - val_acc: 0.5495

Epoch 00001: val_loss did not improve from 1.34828
[1.2239877223968505, 0.593, 1.419096088409424, 0.5495, 1.16]
Done training  16000  images in Epoch :  1
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 1.3466 - acc: 0.5960 - val_loss: 1.7428 - val_acc: 0.4730

Epoch 00001: val_loss did not improve from 1.34828
[1.3465891666412353, 0.596, 1.742786916732788, 0.473, 1.17]
Done training  17000  image

...

1000/1000 [==============================] - 26s 26ms/step - loss: 0.9780 - acc: 0.6920 - val_loss: 1.3256 - val_acc: 0.5820

Epoch 00001: val_loss did not improve from 1.27917
[0.9779946870803833, 0.692, 1.3255700445175171, 0.582, 4.03]
Done training  3000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 26s 26ms/step - loss: 0.9687 - acc: 0.6850 - val_loss: 1.3471 - val_acc: 0.5815

Epoch 00001: val_loss did not improve from 1.27917
[0.9686535620689392, 0.685, 1.3470993938446045, 0.5815, 4.04]
Done training  4000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 0.9963 - acc: 0.6910 - val_loss: 1.2937 - val_acc: 0.5860

Epoch 00001: val_loss did not improve from 1.27917
[0.9963301801681519, 0.691, 1.293684118270874, 0.586, 4.05]
Done training  5000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samp

Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 0.9926 - acc: 0.6720 - val_loss: 1.3466 - val_acc: 0.5765

Epoch 00001: val_loss did not improve from 1.27614
[0.992623249053955, 0.672, 1.346632872581482, 0.5765, 4.27]
Done training  27000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 0.9728 - acc: 0.6840 - val_loss: 1.3287 - val_acc: 0.5845

Epoch 00001: val_loss did not improve from 1.27614
[0.9728166580200195, 0.684, 1.3286583108901977, 0.5845, 4.28]
Done training  28000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 0.9749 - acc: 0.6960 - val_loss: 1.3045 - val_acc: 0.5890

Epoch 00001: val_loss did not improve from 1.27614
[0.9748989343643188, 0.696, 1.3044872360229491, 0.589, 4.29]
Done training  29000  imag

Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 0.8573 - acc: 0.7200 - val_loss: 1.3075 - val_acc: 0.5775

Epoch 00001: val_loss did not improve from 1.27277
[0.8573343591690064, 0.72, 1.3075324344635009, 0.5775, 4.51]
Done training  51000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 0.9187 - acc: 0.7140 - val_loss: 1.3013 - val_acc: 0.5865

Epoch 00001: val_loss did not improve from 1.27277
[0.9187222671508789, 0.714, 1.3013496990203857, 0.5865, 4.52]
Done training  52000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 26s 26ms/step - loss: 0.8744 - acc: 0.7150 - val_loss: 1.3113 - val_acc: 0.5790

Epoch 00001: val_loss did not improve from 1.27277
[0.8743719310760498, 0.715, 1.3112635927200318, 0.579, 4.53]
Done training  53000  ima

1000/1000 [==============================] - 27s 27ms/step - loss: 1.0027 - acc: 0.6750 - val_loss: 1.3324 - val_acc: 0.5715

Epoch 00001: val_loss did not improve from 1.27277
[1.0027115097045898, 0.675, 1.3323939666748046, 0.5715, 4.75]
Done training  75000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 1.0742 - acc: 0.6440 - val_loss: 1.3377 - val_acc: 0.5680

Epoch 00001: val_loss did not improve from 1.27277
[1.0742319202423096, 0.644, 1.3377148294448853, 0.568, 4.76]
Done training  76000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 0.9992 - acc: 0.6820 - val_loss: 1.3165 - val_acc: 0.5815

Epoch 00001: val_loss did not improve from 1.27277
[0.9992247562408447, 0.682, 1.3164555568695069, 0.5815, 4.77]
Done training  77000  images in Epoch :  4
Train on 1000 samples, validate on 2000

1000/1000 [==============================] - 27s 27ms/step - loss: 0.9332 - acc: 0.7100 - val_loss: 1.3125 - val_acc: 0.5825

Epoch 00001: val_loss did not improve from 1.26873
[0.9332143840789795, 0.71, 1.3125338668823243, 0.5825, 4.99]
Done training  99000  images in Epoch :  4
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 0.9214 - acc: 0.7120 - val_loss: 1.3174 - val_acc: 0.5825

Epoch 00001: val_loss did not improve from 1.26873
[0.9214005088806152, 0.712, 1.3174047737121581, 0.5825, 5.0]
Done training  100000  images in Epoch :  4
Done training 100000 images, Epoch  4   -------------


In [12]:
net_final.save('C:/Users/cloudadavncenew0709/Desktop/MobileNet_Condition01_03.h5')
print('Saved Succeed--------------')

Saved Succeed--------------


# 計算top-1、top-3 accuracy

In [13]:
for layer in net_final.layers:
    layer.trainable = False

In [14]:
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

net_final.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics = ['accuracy', top3_acc])

In [15]:
for i in range(2):#訓練5週期
    X = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/imgonehot_"+str((i+1)*1000)+".npy" )
    Y = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/labelonehot_"+str((i+1)*1000)+".npy" )
    net_final.fit(x = X, y = Y,
                    epochs = 1,
                    validation_data=(V1, V2),
                    verbose = 1)

Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 17s 17ms/step - loss: 0.9077 - acc: 0.7170 - top3_acc: 0.9110 - val_loss: 1.3174 - val_acc: 0.5825 - val_top3_acc: 0.8440
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 15s 15ms/step - loss: 0.9046 - acc: 0.7070 - top3_acc: 0.9180 - val_loss: 1.3174 - val_acc: 0.5825 - val_top3_acc: 0.8440


# 載入save_best_model

In [19]:
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
    model = load_model('C:/Users/cloudadavncenew0709/Desktop/MobileNet_Condition01_SaveBest_03.h5')

In [20]:
for layer in model.layers:
    layer.trainable = False

In [21]:
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics = ['accuracy', top3_acc])

In [22]:
for i in range(2):
    X = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/imgonehot_"+str((i+1)*1000)+".npy" )
    Y = np.load( "C:/Users/cloudadavncenew0709/Desktop/numpydataKeras/labelonehot_"+str((i+1)*1000)+".npy" )
    model.fit(x = X, y = Y,
              epochs = 1,
              validation_data=(V1, V2),
              verbose = 1)

Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 18s 18ms/step - loss: 0.9600 - acc: 0.6900 - top3_acc: 0.9020 - val_loss: 1.2687 - val_acc: 0.5835 - val_top3_acc: 0.8515
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 15s 15ms/step - loss: 0.9070 - acc: 0.7160 - top3_acc: 0.9220 - val_loss: 1.2687 - val_acc: 0.5835 - val_top3_acc: 0.8515
